### Let's develop a chatbot based on groq llms using langchain and deploy it on haggingface 

In [13]:
import os 
import warnings
from langchain_groq import ChatGroq
from langchain_community.llms import HuggingFaceEndpoint

warnings.filterwarnings("ignore")

In [5]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

### Prompt Templates nd langchain chains

In [45]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [43]:
prompt_template = PromptTemplate(input_variables = ["word"],
                                 template = "What are 5 synonyms of word {word}")
prompt_template.format(word = "impossible")

'What are 5 synonyms of word impossible'

In [48]:
chain = LLMChain(llm = groq_llm, prompt = prompt_template)
synonyme_response = chain.run("impossible")
print (synonyme_response)

Here are 5 synonyms for the word "impossible":

1. Unfeasible
2. Inconceivable
3. Unthinkable
4. Unachievable
5. Unattainable


In [49]:
print (chain.run("possible"))

Here are 5 synonyms of the word "possible":

1. Feasible
2. Probable
3. Viable
4. Plausible
5. Likely


### Combining multiple chains using simple sequential chain 

In [18]:
import os 
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_groq import ChatGroq
from langchain.chains import SimpleSequentialChain

In [6]:
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant",
    temperature = 0.6,
)

In [10]:
first_prompt = PromptTemplate(input_variables =["word"], 
                             template = "What are one synonym of word {word}, return the synonyme and nothing else" )
first_chain = LLMChain(llm = groq_llm, prompt = first_prompt)

In [14]:
first_chain.run(word = "impossible")

'Infeasible'

In [20]:
second_prompt = PromptTemplate(input_variables = ["synonyme"],
                                template = "What is the translation of this word {synonyme} in french")
second_chain = LLMChain(llm = groq_llm, prompt = second_prompt)

In [17]:
print(second_chain.run(synonyme = "Infeasible"))

The translation of the word "Infeasible" in French is "Impossible à réaliser" or "Inexécutable". However, a more common and idiomatic translation would be "Inabordable" or "Non réalisable".

Alternatively, you can also use the word "Iréalisable" which is more literal and direct translation of the word "Infeasible".

It's worth noting that the context in which the word is used can affect the choice of translation.


In [22]:
#Sequantial chain 
simple_sequence_chain = SimpleSequentialChain(chains = [first_chain, second_chain])

In [24]:
output = sequence_chain.run("nice")
print (output)

The translation of the word "Friendly" in French is:

- Amical (masculine)
- Amicale (feminine)

However, if you're referring to a friendly atmosphere or a friendly place, you can also use:

- Chaleureux (warm, welcoming)
- Accueillant (welcoming, hospitable)
- Amical (friendly, in a general sense)

If you're talking about a friendly game or competition, you can use:

- Amical (friendly, in a sporting context)

Keep in mind that the context and the situation will influence the correct translation and usage of the word.


### Sequential chains 

In [36]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain

In [48]:
first_template = PromptTemplate(input_variables = ["proverb"], 
                                template = "Complete this proverb {proverb}, return the complete proverb only nothin more.")
first_chain = LLMChain(llm = groq_llm, prompt = first_template, output_key ="proverb_completed")

In [27]:
print (first_chain.run("He who plays with the sword"))

He who plays with the sword, must be prepared to be wounded.


In [49]:
second_template = PromptTemplate (input_variables = ["proverb_completed"], 
                                  template = """
                                                Translate this proverbe <{proverb_completed}> to french, 
                                                return only one translation and nothing more.
                                             """)
second_chain = LLMChain(llm = groq_llm, prompt = second_template, output_key = "translation")

In [35]:
print (second_chain.run("He who plays with the sword, must be prepared to be wounded."))

The translation of "He who plays with the sword, must be prepared to be wounded" in French is:

"Qui joue avec l'épée, doit être prêt à se faire blesser."

However, a more idiomatic translation would be:

"Qui se fait l'amitié de l'épée, doit se faire la guerre."

This translation uses a more poetic and idiomatic expression, which conveys the same idea as the original phrase.


In [50]:
sequential_chain = SequentialChain(chains = [first_chain, second_chain],
                input_variables = ["proverb"],
                output_variables = ["proverb_completed", "translation"])

In [51]:
sequential_chain({"proverb": "He who plays with the sowrd"})

{'proverb': 'He who plays with the sowrd',
 'proverb_completed': 'He who plays with the sword, gets cut.',
 'translation': '"Qui se livre à l\'épée, se fait couper."'}

### Chat models, System messages 

In [52]:
from langchain_core.messages import SystemMessage, HumanMessage

In [55]:
messages = [(SystemMessage("You are an expert in proverbs, user will give you one theme, you have to present 3 proverbs that corresponds to that theme")),
            (HumanMessage("Courage"))]
output = groq_llm.invoke(messages)
print(output.content)

Here are three proverbs that correspond to the theme of "Courage":

1. **"Courage is not the absence of fear, but rather the judgment that something else is more important than fear."** - Ambrose Redmoon

This proverb emphasizes the importance of courage in the face of fear. It suggests that true courage involves prioritizing what's important over fear, rather than simply ignoring it.

2. **"Fall seven times, stand up eight."** - Japanese proverb

This proverb conveys the idea that courage involves perseverance and resilience in the face of adversity. It suggests that even when we face setbacks, we should keep getting back up and trying again.

3. **"The lion is tamed not by a whip, but by a gentle voice."** - Latin proverb

This proverb suggests that courage can be achieved through gentle persuasion and encouragement, rather than through force or coercion. It emphasizes the importance of kindness and compassion in building courage and confidence.

I hope these proverbs inspire you to 

### Message persistence

In [26]:
import os
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, StateGraph, MessagesState
from langchain_groq import ChatGroq

In [27]:
#Model
groq_llm = ChatGroq(
    groq_api_key = os.environ["GROQ_API_KEY"],
    model_name = "llama-3.1-8b-instant", #Check groq website for other llm ids
    temperature = 0.6 
)

In [35]:
#Worflow 
workflow = StateGraph(state_schema = MessagesState)
#define how to call the model 
def call_model(state: MessagesState):
    response = groq_llm.invoke(state["messages"])
    return {"messages": response}
#node and edges
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [36]:
#Set configuration
config_1 = {"configurable": {"thread_id": "chat_1"}}
config_2 = {"configurable": {"thread_id": "chat_2"}}

In [38]:
def invoke_app(query : str, config):
    response = app.invoke({"messages": [HumanMessage(query)]}, config)
    return response["messages"][-1].content

In [39]:
query_1 = "Hello, it is 4 o clock pm, here in morocco, what time could be in new york, America ?"
invoke_app(query_1, config_1)

================================== Ai Message ==================================

Since Morocco is in the Western European Time (WET) or Central European Time (CET) zone, which is UTC+0 or UTC+1, and New York is in the Eastern Time Zone (ET), which is UTC-5, we need to consider the time difference.

Assuming Morocco is in UTC+0 (winter) and New York is in standard time (UTC-5), the time difference would be 5 hours. 

If it's 4 o'clock pm in Morocco, 

- It would be 11 am in New York (4 pm + 5 hours).

However, Morocco is in UTC+1 (summer) from March to October and New York is in daylight saving time from March to November, so let's consider that scenario.

If Morocco is in UTC+1 and New York is in daylight saving time (UTC-4), the time difference would be 6 hours.

- If it's 4 o'clock pm in Morocco, it would be 10 am in New York (4 pm + 6 hours).

Please note that Morocco and New York may not be in the same time zone or daylight saving time at the same time.


In [40]:
query_2 = "What would be the time in morocco in 2 hours, give the time only, nothing more."
invoke_app(query_2, config_1)

================================== Ai Message ==================================

6 pm


In [42]:
invoke_app(query_2, config_2)

================================== Ai Message ==================================

UTC+0 

Current time: 12:00 
In 2 hours: 14:00


### test

In [47]:
from langchain_core.messages import BaseMessage
from typing import TypedDict, List

In [87]:
groq_llm = ChatGroq( groq_api_key = os.environ["GROQ_API_KEY"], 
                        model_name = "llama-3.1-8b-instant", 
                        temperature = 0.6 ) 

class MessagesState(TypedDict):
    messages: List[BaseMessage]

#Build the app using langraph 
workflow = StateGraph(state_schema = MessagesState) 
# 
def call_model(state:MessagesState): 
	response = groq_llm.invoke(state["messages"]) 
	return {"messages": response} 

#add nodes and edges 
workflow.add_edge(START, "model") 
workflow.add_node("model", call_model) 
#Memory 
memory = MemorySaver() 
app = workflow.compile(checkpointer = memory) 

In [88]:
config_3 = {"configurable" : {"thread_id": "111"}}
config_3

{'configurable': {'thread_id': '111'}}

In [89]:
print("config type :", type(config_3))
print ("config: ", config_3)
thread_id = config_3["configurable"]["thread_id"]
print ("thread :", thread_id)
# Retrieve prior state (conversation history) from the memory

config type : <class 'dict'>
config:  {'configurable': {'thread_id': '111'}}
thread : 111


In [91]:
app.get_state(config_3).values

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '111'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [92]:
try:
    previous_state = app.get_state(config_3)
    messages = previous_state["messages"]
except Exception:
    messages = []
messages 

[]

In [93]:
query = "Hello"
messages.append(HumanMessage(content=query))
messages 

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [94]:
# Invoke the app with the full message history
response = app.invoke({"messages": messages}, config_3)

In [95]:
response["messages"].content

'Hello. How can I assist you today?'

In [96]:
app.get_state(config_3).values["messages"]

AIMessage(content='Hello. How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46, 'completion_time': 0.011811121, 'prompt_time': 0.00155279, 'queue_time': 0.088089074, 'total_time': 0.013363911}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--5fac31c5-5d71-4375-bf20-2fcaec4ee55a-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [97]:
messages.append

[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]